In [0]:
import datetime
date_ = datetime.datetime.now().strftime("%Y-%m-%d")
bronze_path = f"abfss://projectfiles@practicestorageacc0.dfs.core.windows.net/videostream/bronze/{date_}"
df_bronze = spark.read.format("parquet").load(bronze_path)


In [0]:
from pyspark.sql.functions import col, from_unixtime, to_timestamp, lit, when, expr
numeric_pattern = r'^\d+(\.\d+)?$'
df_silver = df_bronze \
    .filter(col("user_id").isNotNull())\
    .withColumn("load_date", lit(date_))\
    .withColumn("viewers_count", when(col("message").rlike(numeric_pattern),  col("message").cast("int")))\
    .withColumn("chat_message", when(~col("message").rlike(numeric_pattern), col("message")))\
    .drop("message")

df_silver = df_silver.withColumn("timestamp", to_timestamp(col("timestamp").cast("timestamp"), "yyyy-MM-dd HH:mm:ss")).withColumnRenamed("timestamp", "created_at")\
    .withColumn("user_id", expr("substring(base64(sha1(user_id)), 1, 10)"))\
    .withColumn("video_id", expr("substring(base64(sha1(video_id)),1, 10)"))\
    

# df_silver.display()

In [0]:

silver_path = f"abfss://projectfiles@practicestorageacc0.dfs.core.windows.net/videostream/silver/{date_}/"
df_silver.write.mode("overwrite")\
    .format("parquet")\
    .save(silver_path)